In [1]:
import datetime
from pytz import timezone, utc

## timezoneを利用する場合

### timezoneを指定したdatetime

datetimeの値を固定してtimezoneを指定する場合は，datetime.datetimeのtzinfoにpytzのtimezoneオブジェクトを渡す方法と，timezoneオブジェクトのlocalizeメソッドを利用する方法の二つがある．しかし，tzinfoで指定してしまうと，時差は9時19分となるので，timezoneクラスのlocalizeメソッドを利用しよう．
[参考](https://qiita.com/higitune/items/0ca244373d380cf1c060)
実はもう一つの方法がある(datetme.datetimeのreplaceメソッドをつかう)が，こちらも時差は9時19分となってしまう．

タイムゾーンありの時間表記は，現地時間＋utcからの時差　となる．

In [2]:
datetime1 = datetime.datetime(2020, 10, 31, 12, 0, 0)
print("datetime1:",datetime1)
print("datetime1 type:",type(datetime1))
print("datetime1 timezone:", datetime1.tzname())
print("datetime1 unixtime:", datetime1.timestamp())

jst_timezone = timezone('Asia/Tokyo')
datetime2 = datetime.datetime(2020, 10, 31, 12, 0, 0, tzinfo=jst_timezone)
print("datetime2:",datetime2)
print("datetime2 type:",type(datetime2))
print("datetime2 timezone:", datetime2.tzname())
print("datetime2 unixtime:", datetime2.timestamp())

datetime3 = jst_timezone.localize(datetime.datetime(2020, 10, 31, 12, 0, 0))
print("datetime3:",datetime3)
print("datetime3 type:",type(datetime3))
print("datetime3 timezone:", datetime3.tzname())
print("datetime3 unixtime:", datetime3.timestamp())

datetime1: 2020-10-31 12:00:00
datetime1 type: <class 'datetime.datetime'>
datetime1 timezone: None
datetime1 unixtime: 1604113200.0
datetime2: 2020-10-31 12:00:00+09:19
datetime2 type: <class 'datetime.datetime'>
datetime2 timezone: LMT
datetime2 unixtime: 1604112060.0
datetime3: 2020-10-31 12:00:00+09:00
datetime3 type: <class 'datetime.datetime'>
datetime3 timezone: JST
datetime3 unixtime: 1604113200.0


上を確認すると，naiveなdatetimeオブジェクトはタイムゾーンに日本を指定した値と一致していることが分かる．つまり，naiveなdatetimeオブジェクトは通常は現地時間になってしまう．これはunixtimeからdatetime.fromtimestampを用いてdatetimeオブジェクトを作成しても同じ結果になる．

In [3]:
add_datetime1 = datetime1 + datetime.timedelta(days=1)
print(add_datetime1)
add_datetime2 = datetime3 + datetime.timedelta(days=1)
print(add_datetime2)

2020-11-01 12:00:00
2020-11-01 12:00:00+09:00


上のようにdatetime.timedeltaを利用して和もとれる

### 同時刻(同じunixtime)のタイムゾーンの異なるdatetimeを取得

タイムゾーンの指定されたdatetimeが与えらえたとき，他の地域におけるその時刻のdatetimeを取得したいときがある．そのようなときは，datetime.datetime.astimezoneメソッドを利用する．

In [4]:
hk_timezone = timezone("Asia/Hong_Kong")
datetime4 = datetime3.astimezone(hk_timezone)
print("datetime4:",datetime4)
print("datetime4 type:",type(datetime4))
print("datetime4 timezone:", datetime4.tzname())
print("datetime4 unixtime:", datetime4.timestamp())

datetime4: 2020-10-31 11:00:00+08:00
datetime4 type: <class 'datetime.datetime'>
datetime4 timezone: HKT
datetime4 unixtime: 1604113200.0


astimezoneはnaiveなdatetimeでも利用できる．その場合，パソコンのローカルなタイムゾーンが指定される．

###  同時刻(同じunixtime)のUTCのnaiveなdatetimeを取得

sqliteではnaiveなdatetimeしか利用できないっぽいので，データベースにはUTCで保存する．そこで，タイムゾーンが指定されたdatetimeから，同じunix時刻を持つUTC時刻に対応したnaiveなdatetimeを取得する．やり方は，以下のように行う．
- awareなdatetimeからtimestampメソッドを利用してunix時刻を取得
- unix時刻からdatetime.datetime.utcfromtimestampを利用してdatetimeを取得

In [5]:
datatime3_unix = datetime3.timestamp()
utc_datetime3 = datetime.datetime.utcfromtimestamp(datatime3_unix)
print("utc_datetime3:",utc_datetime3)
print("utc_datetime3 type:",type(utc_datetime3))
print("utc_datetime3 timezone:", utc_datetime3.tzname())
print("utc_datetime3 unixtime", utc_datetime3.timestamp())

utc_datetime3: 2020-10-31 03:00:00
utc_datetime3 type: <class 'datetime.datetime'>
utc_datetime3 timezone: None
utc_datetime3 unixtime 1604080800.0


ここで，unixtimeがdatetime3と異なっていることが分かる．これは，naiveなdatetimeのタイムゾーンが日本になってしまうことが原因である．このnaiveなdatetimeをutcにlocalizしてしまえば，unixtimeが一致するはずである．

In [6]:
utc_timezone = timezone("UTC")
localized_utc_datetime3 = utc_timezone.localize(utc_datetime3)
print("localized_utc_datetime3:",localized_utc_datetime3)
print("localized_utc_datetime3 type:",type(localized_utc_datetime3))
print("localized_utc_datetime3 timezone:", localized_utc_datetime3.tzname())
print("localized_utc_datetime3 unixtime:", localized_utc_datetime3.timestamp())

localized_utc_datetime3: 2020-10-31 03:00:00+00:00
localized_utc_datetime3 type: <class 'datetime.datetime'>
localized_utc_datetime3 timezone: UTC
localized_utc_datetime3 unixtime: 1604113200.0


### 同じdatetimeの値を持つタイムゾーンの異なるdatetimeを取得 

ほとんど使い道はないだろうが，同じdatetimeの値を持つタイムゾーンの異なるdatetimeオブジェクト取得する．unix時刻は異なることに注意．<- 使い道はあった表示の時

In [8]:
without_tz_datetime4 = datetime4.replace(tzinfo=None)
print("without_tz_datetime4:", without_tz_datetime4)
print("without_tz_datetime4 type:", type(without_tz_datetime4))
print("without_tz_datetime4 timezone",without_tz_datetime4.tzname())
print("without_tz_datetime4 unixtime:",without_tz_datetime4.timestamp())

without_tz_datetime4: 2020-10-31 11:00:00
without_tz_datetime4 type: <class 'datetime.datetime'>
without_tz_datetime4 timezone None
without_tz_datetime4 unixtime: 1604109600.0


ちなみに，astimezone(None)とすると，日本のタイムゾーンが設定されるので注意

### タイムゾーン名の取得 

存在しない場合，Noneが返る．

In [9]:
print(datetime1.tzname())
print(datetime2.tzname())

None
LMT


In [10]:
print(datetime1.tzinfo)
print(datetime3.tzinfo)

None
Asia/Tokyo


実は，tzinfoはtzinfoクラスであるので，timezoneのコンストラクタにはstr(tzinfo)として文字列にしてから渡す．

### システムローカルなタイムゾーンを取得

In [11]:
import dateutil
print(dateutil.tzlocal())

AttributeError: module 'dateutil' has no attribute 'tzlocal'

## pandasにおける取り扱い 

seriesでも扱いは同じだと思うので，timestampをインデックスとして利用する場合を考える．pandasではdatetimeをインデックスとして利用する場合はDateTimeIndexクラスを用いるが，その際タイムゾーンも維持される

In [12]:
import pandas as pd

In [13]:
datetime_list1 = [datetime.datetime(2020, 10, 31, 12, 0, 0),
                  datetime.datetime(2020, 10, 31, 13, 0, 0),
                 ]

df_list = [[0, 1], [2, 3]]

df1 = pd.DataFrame(df_list, index=datetime_list1)
df1

,0,1
2020-10-31 12:00:00,0,1
2020-10-31 13:00:00,2,3


### timezoneを指定する

In [14]:
datetime_list2 = [jst_timezone.localize(datetime.datetime(2020, 10, 31, 12, 0, 0)),
                  jst_timezone.localize(datetime.datetime(2020, 10, 31, 13, 0, 0)),
                 ]

df_list = [[0, 1], [2, 3]]

df2 = pd.DataFrame(df_list, index=datetime_list2)
df2

,0,1
2020-10-31 12:00:00+09:00,0,1
2020-10-31 13:00:00+09:00,2,3


pandas.DateTimeIndexのtz_localizeメソッドを利用することで，タイムゾーンをpandas側で指定できる．

In [15]:
df3 = pd.DataFrame(df_list, index=datetime_list1)
df3.index = df3.index.tz_localize("Asia/Tokyo")
df3

,0,1
2020-10-31 12:00:00+09:00,0,1
2020-10-31 13:00:00+09:00,2,3


### 同時刻（同じunixtime）のタイムゾーンの異なるdatetimeindexを取得 

pandas.DateTimeIndexのtz_convertメソッドを利用する．これはdatetime.datetime.astimezoneに相当する．

In [16]:
df3.index = df3.index.tz_convert("Asia/Hong_Kong")
df3

,0,1
2020-10-31 11:00:00+08:00,0,1
2020-10-31 12:00:00+08:00,2,3


###  タイムゾーン名の取得

存在していない場合，Noneが返る

In [17]:
print(str(df1.index.tzinfo))
print(str(df3.index.tzinfo))

None
Asia/Hong_Kong


### datetimeに変換する場合

そのままto_pydatetimeで変更してしまうと，awareで返ってきてしまう．

In [18]:
index_list = df3.index.to_pydatetime()
index_list

array([datetime.datetime(2020, 10, 31, 11, 0, tzinfo=<DstTzInfo 'Asia/Hong_Kong' HKT+8:00:00 STD>),
       datetime.datetime(2020, 10, 31, 12, 0, tzinfo=<DstTzInfo 'Asia/Hong_Kong' HKT+8:00:00 STD>)],
      dtype=object)

### 同時刻のUTCのnaiveなdatetimeを取得 

tz_convertでNoneを指定することでNaiveなutcのdatetimeが得られる．これは，UTCの時間になる!

In [19]:
df3.index.tz_convert(None).to_pydatetime()

array([datetime.datetime(2020, 10, 31, 3, 0),
       datetime.datetime(2020, 10, 31, 4, 0)], dtype=object)

In [68]:
df3.index = df3.index.tz_convert(None)
print(df3.index.tzinfo)

None


In [69]:
df3.index = df3.index.tz_convert(None)
print(df3.index.tzinfo)

TypeError: Cannot convert tz-naive timestamps, use tz_localize to localize

Noneを指定してnaiveなdatatimeにした場合，もう一度tz_convertするとエラーが出るので注意！

### 同じdatetimeの値を持つタイムゾーンの異なるdatetimeを取得 

使い道がないと思ったが，表示するのにnaiveである必要がある場合に使える．tz_localizeにNoneを指定する．

In [24]:
df3 = pd.DataFrame(df_list, index=datetime_list1)
df3.index = df3.index.tz_localize("Asia/Tokyo")
print(df3.index)

df3.index = df3.index.tz_localize(None)
print(df3.index)

DatetimeIndex(['2020-10-31 12:00:00+09:00', '2020-10-31 13:00:00+09:00'], dtype='datetime64[ns, Asia/Tokyo]', freq=None)
DatetimeIndex(['2020-10-31 12:00:00', '2020-10-31 13:00:00'], dtype='datetime64[ns]', freq=None)


### sqliteにおける取り扱い 

In [16]:
from pathlib import Path
import sqlite3
from contextlib import closing

In [17]:
db_path = Path("datetime.db")
def make_conn():
    conn = sqlite3.connect(db_path,
                           detect_types=sqlite3.PARSE_DECLTYPES|sqlite3.PARSE_COLNAMES
                          )
    return conn

#### sqlite3パッケージを利用してinsert 

In [18]:
datetime_list1 = [[datetime.datetime(2020, 10, 31, 0, 0, 0),],
                  [datetime.datetime(2020, 10, 31, 0, 1, 0),]
                 ]


with closing(make_conn()) as conn:
    c = conn.cursor()
    c.execute("create table times1 (datetime timestamp)")
    c.executemany("insert into times1(datetime) values(?)",datetime_list1)
    conn.commit()
    
    c.execute("select * from times1")
    print(c.fetchall())
    c.execute("drop table times1")

[(datetime.datetime(2020, 10, 31, 0, 0),), (datetime.datetime(2020, 10, 31, 0, 1),)]


timezone指定をしてしまうと，エラーが出てしまう

In [77]:
datetime_list2 = [[jst_timezone.localize(datetime.datetime(2020, 10, 31, 12, 0, 0)),],
                  [jst_timezone.localize(datetime.datetime(2020, 10, 31, 13, 0, 0)),]
                 ]


with closing(make_conn()) as conn:
    c = conn.cursor()
    c.execute("create table times2 (datetime timestamp)")
    c.executemany("insert into times2(datetime) values(?)",datetime_list2)
    conn.commit()
    
    c.execute("select * from times2")
    print(c.fetchall())
    c.execute("drop table times2")

ValueError: invalid literal for int() with base 10: b'00+09'

In [76]:
with closing(make_conn()) as conn:
    c = conn.cursor()
    c.execute("drop table times2")

### まとめ 

- local timeが必要な場合はaware なdatetimeを使うと決めておく．その時naiveなdatetimeのタイムゾーンは自動的に日本になることに注意
- つまり，naiveなdatetimeのunix時刻は，タイムゾーンが東京都指定されたawareなdatetimeのものと一致する
- spliteはnaiveなutcしかinsertできないので，必ずutcを入れる．